### Computing errors associated with the choice of weight function for the RM
#### Used for Table 2 

Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from piston_Lagrangian import setup
from utils import max_error

Compute a reference solution

In [ ]:
p = 20 #Number of  periods
p_domain = 30 #Number of periods in the domain
xmax1 = p_domain*2*np.pi
N = 250 #Cells per period at final time
mx1 = p_domain*N#int(N*(xmax1/p)) #Number of grid points
M = 0.4 
piston_freq=1.
tfinal = p*2*np.pi
piston_problem1 = setup(tfinal=tfinal, xmax=xmax1, mx=mx1, M=M, CFL=0.8,limiting=1,solver_type='sharpclaw',
order=2, time_integrator='Heun', euler_RS='euler', nout=100,
piston_freq=piston_freq)
piston_problem1.verbosity = 0
piston_problem1.run()
piston_problem1.solver.__del__() #Deallocation of Fortran arrays

In [ ]:
#Combinations of s and d
combination_names = ["RM "+r"\Gamma_A","RM "+r"\Gamma_B","RM-M "+r"\Gamma_A","RM-M "+r"\Gamma_B"]
methods=["RM","RM-M"]
list_functions = ["Engsig-Karup2","Mayer"]
list_combinations = [(0,0),(0,1),(1,0),(1,1)]
matrix_filters = [False,True]
far_field_damping = False #Exponential far_field_damping (w. source term)
scalar_far_field =False
relaxation_method = True
integral_source = False
euler_RS = 'euler'
solver_class = 'sharpclaw_custom'
Strang = False
before_stage = False

Computing error

In [ ]:
def comparison_RM_weight_funcs(stop_slowing=25*np.pi ,stop_absorbing=25*np.pi,
                                      list_functions=list_functions, matrix_filters=matrix_filters,
                                      list_combinations=list_combinations):
    p = 20 #Number of  periods
    p_domain = 30 #Number of periods in the domain
    xmax2 = p_domain*2*np.pi
    N = 250#200 #Cells per period at final time
    mx2 = p_domain*N#int(N*(xmax1/p)) #Number of grid points
    M = 0.4 
    tfinal = p*2*np.pi#p*2*np.pi
    start_slowing = 20*np.pi
    stop_slowing = stop_slowing
    start_absorbing = 20*np.pi
    stop_absorbing = stop_absorbing
    local_list_errors = []
    for i in range(len(list_combinations)):
        idx1, idx2 = list_combinations[i]
        matrix_filter, weight_fun = matrix_filters[idx1],list_functions[idx2]
        piston_problem2 = setup(tfinal=tfinal, xmax=xmax2, mx=mx2, M=M, CFL=0.8,limiting=1,nout=100,
                        solver_type=solver_class, order=2, time_integrator='Heun',
                        euler_RS=euler_RS, 
                        relaxation_method=relaxation_method,
                        far_field_damping=far_field_damping,
                        matrix_filter=matrix_filter,
                        start_slowing=start_slowing, stop_slowing=stop_slowing,
                        start_absorbing=start_absorbing, stop_absorbing=stop_absorbing,
                        piston_freq=piston_freq,
                        sigma_damping_type=weight_fun, 
                        scalar_far_field=scalar_far_field,
                        before_stage=before_stage,
                        adaptiveRM=False,
                        Strang=Strang)
        piston_problem2.verbosity = 0
        piston_problem2.run()
        piston_problem2.solver.__del__() #Deallocation of Fortran arrays
        indx = np.where(piston_problem2.grid.x.centers<=start_slowing)
        local_list_errors.append(max_error(piston_problem1,piston_problem2, indx=indx))
    return local_list_errors


In [ ]:
list_errors = []
for i in [0.25,0.5,1.,2.]:
    list_errors.append(comparison_RM_weight_funcs(stop_slowing=(20+i)*np.pi, stop_absorbing=(20+i)*np.pi))

In [ ]:
list_errors

In [ ]:
from tabulate import tabulate
print(tabulate(list_errors, tablefmt="latex", floatfmt=".2e", headers=combination_names))
